In [5]:
import pandas as pd
import numpy as np
from collections import defaultdict

# Sample data
data = {
    'UserID': [1, 1, 1, 2, 2, 3, 3, 3, 4, 4, 5, 5],
    'MovieID': [101, 102, 103, 101, 104, 102, 103, 105, 104, 105, 101, 105],
    'Rating': [5, 4, 3, 4, 5, 2, 4, 5, 3, 4, 5, 4]
}
df = pd.DataFrame(data)

# Create user-item matrix
user_item_matrix = df.pivot_table(index='UserID', columns='MovieID', values='Rating').fillna(0)

# Calculate cosine similarity between users
def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b)

# Find similar users
def find_similar_users(user_id, matrix, k=3):
    similarities = []
    for uid in matrix.index:
        if uid != user_id:
            sim = cosine_similarity(matrix.loc[user_id], matrix.loc[uid])
            similarities.append((uid, sim))
    similarities.sort(key=lambda x: x[1], reverse=True)
    return similarities[:k]

# Generate recommendations
def recommend_movies(user_id, n=3):
    similar_users = find_similar_users(user_id, user_item_matrix)
    user_movies = set(df[df['UserID'] == user_id]['MovieID'])
    
    recommendations = defaultdict(float)
    for uid, similarity in similar_users:
        for _, row in df[df['UserID'] == uid].iterrows():
            if row['MovieID'] not in user_movies:
                recommendations[row['MovieID']] += row['Rating'] * similarity
    
    return sorted(recommendations.items(), key=lambda x: x[1], reverse=True)[:n]

# Test the system
user_id = 2
print(f"Top recommendations for User {user_id}:")
for movie, score in recommend_movies(user_id):
    print(f"Movie {movie} (Recommendation Score: {score:.2f})")

Top recommendations for User 2:
Movie 105 (Recommendation Score: 3.83)
Movie 102 (Recommendation Score: 1.77)
Movie 103 (Recommendation Score: 1.33)
